In [1]:
import konlpy
from konlpy.tag import Kkma
import pandas as pd
import numpy as np
import re
import glob
import pickle
import pandas as pd

In [2]:
df_train = pd.read_csv('./dataset/naver_train.txt', delimiter='\t', keep_default_na=False)
print(df_train.shape)
df_train.head(10)

(150000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [3]:
def resub(t_list):
    return_list=[]
    for t in t_list:
        return_list.append(re.sub('[`~!@#\;:/*+$%^&*(),.ㅋ]','',t))
                           
    return return_list         

In [4]:
text_list = resub([row for row in df_train['document'][:]])
text_lable = [row for row in df_train['label'][:]]
train_data=[]
for i in range(len(text_lable)):
    train_data.append((text_list[i], text_lable[i]))
train_data[-12]

('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', 0)

In [5]:
train_data[1][1]

1

In [6]:
f = open('./output.csv', 'w', encoding='utf-8')

In [7]:
#for i in range(len(text_lable)):
for i in range(50):
    f.write(train_data[i][0]+','+str(train_data[i][1])+'\n')
f.close()

In [8]:
f.close()

In [3]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchtext import data
#Field,Iterator,TabularDataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from konlpy.tag import Kkma
tagger = Kkma()

In [6]:
ID =data.Field(sequential=False, use_vocab=False)
TEXT = data.Field(tokenize=tagger.morphs, use_vocab=True, batch_first=True)
LABEL = data.Field(sequential=False, use_vocab=False)

train_data = data.TabularDataset(path="./dataset/naver_train_1.txt",
                                 format="tsv", skip_header = True,
                                 fields=[('ID',ID), ('TEXT',TEXT),('LABEL',LABEL)])

In [7]:
train_data

In [4]:
# Create Dataset
class CSVDataset(Dataset):
    def __init__(self, path, chunksize, nb_samples):
        self.path = path
        self.chunksize = chunksize
        self.len = nb_samples / self.chunksize

    def __getitem__(self, index):
        x = next(
            pd.read_csv(
                self.path,
                skiprows=index * self.chunksize + 1,  #+1, since we skip the header
                chunksize=self.chunksize,
                names=['id','document','label']))
        x = torch.from_numpy(x.data.values)
        return x

    def __len__(self):
        return self.len

In [5]:
dataset = CSVDataset('data/naver_test.csv', chunksize=10, nb_samples=50000)
loader = DataLoader(dataset, batch_size=10, num_workers=1, shuffle=False)

for batch_idx, data in enumerate(loader):    
    print('batch: {}\tdata: {}'.format(batch_idx, data))

TypeError: 'float' object cannot be interpreted as an integer

In [9]:
print(train_data.examples[3].ID)
print(train_data.examples[3].TEXT)
print(train_data.examples[3].LABEL)
len(train_data)

6825595
['지루', '하', '지', '는', '않', '은데', '완전', '막장', '임', '...', '돈', '주고', '보', '기에', '는', '....']
0


20

In [33]:
TEXT.build_vocab(train_data)

In [34]:
train_loader =  Iterator(train_data, batch_size=5, 
                         device=-1,repeat=False)  # device -1 : cpu, device 0 : 남는 gpu
    

In [57]:
class RNN(nn.Module):
    def __init__(self,input_size, embed_size, 
                       hidden_size, output_size, 
                       num_layers=1, bidirec=False):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirec:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.embed = nn.Embedding(input_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirec)
        self.linear = nn.Linear(hidden_size*self.num_directions,output_size)
        
    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        cell = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        return hidden, cell
    
    def forward(self,inputs):
        """
        inputs : B,T
        """
        embed = self.embed(inputs) # word vector indexing
        hidden, cell = self.init_hidden(inputs.size(0)) # initial hidden,cell
        
        output, (hidden, cell) = self.lstm(embed, (hidden,cell))
        
        # Many-to-One
        hidden = hidden[-self.num_directions:] # (num_directions,B,H)
        hidden = torch.cat([h for h in hidden],1)
        output = self.linear(hidden) # last hidden
        
        return output

In [70]:
model = RNN(len(TEXT.vocab),100,50,1,bidirec=False)
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

In [71]:
model

RNN(
  (embed): Embedding(213, 100)
  (lstm): LSTM(100, 50, batch_first=True)
  (linear): Linear(in_features=50, out_features=1, bias=True)
)

In [72]:
for batch in train_loader:
    model.zero_grad()
    preds = model(batch.TEXT)
    print(preds)
    loss = loss_function(preds, batch.LABEL.type(torch.FloatTensor))
    print(loss.data[0])
    loss.backward()
    optimizer.step()

Variable containing:
 0.1732
-0.0369
 0.1728
 0.1732
 0.1732
[torch.FloatTensor of size 5x1]

0.5004668831825256
Variable containing:
 0.2906
 0.2918
 0.2918
 0.2918
-0.0113
[torch.FloatTensor of size 5x1]

0.3178107738494873
Variable containing:
 0.4168
 0.0278
 0.4168
 0.4168
 0.4168
[torch.FloatTensor of size 5x1]

0.20569118857383728
Variable containing:
 0.5235
 0.5170
-0.0270
 0.5066
 0.5078
[torch.FloatTensor of size 5x1]

0.2081913948059082
